In [16]:
# !pip install pandas fastparquet gliner[gpu]

In [17]:
from gliner import GLiNER
import pandas as pd
import torch 

In [18]:
df = pd.read_parquet("articles.parquet")

In [19]:
df.head()

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,race_and_racism,Yahoo! The 360,1,https://news.yahoo.com/how-do-we-address-racis...,How do we address racism in 'Gone With the Wind'?,2020-06-18,Julia Munslow,“ The 360 ” shows you diverse perspectives on ...,“The 360” shows you diverse perspectives on th...,www.news.yahoo.com,center,zzwPV6NCYsMiDb0a
1,white_house,Newsmax - News,2,https://www.newsmax.com/newsfront/ppe-masks-gl...,China Agrees Not to Restrict PPE Exports White...,2020-04-05,Jason Devaney,China promised it would not restrict exports o...,China promised it would not restrict exports o...,www.newsmax.com,right,zzw8ORfncyrlNR6J
2,coronavirus,MarketWatch,2,https://www.marketwatch.com/story/fool-me-twic...,"Fool me twice? For businesses and consumers, c...",2020-04-16,Andrea Riquier,"As the coronavirus outbreak claims lives , job...","As the coronavirus outbreak claims lives, jobs...",www.marketwatch.com,right,zzmg5VFM53K2PfDX
3,technology,The Verge,0,https://www.theverge.com/2019/11/6/20948547/tw...,"Twitter is rolling out Topics, a way to follow...",2019-11-06,Casey Newton,"Recently , a friend told me he wanted to spend...","Recently, a friend told me he wanted to spend ...",www.theverge.com,left,zzkjinLBHepHRGbe
4,cybersecurity,Bloomberg,1,http://www.bloomberg.com/politics/articles/201...,White House Outlines New Cybersecurity Initiat...,2015-01-12,Ali Elkin,Ahead of President Barack Obama 's visit to th...,SHARE THIS ARTICLE Share Tweet Post Email\n\nP...,www.bloomberg.com,center,zzjFSqASfeSPDsh1


In [20]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = GLiNER.from_pretrained("EmergentMethods/gliner_medium_news-v2.1")
model.to("cuda")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(128004, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): Dropout(p=0.1, inplace=False)
                    (dropout): Dropout(p=0.1, inplace=False)
            

In [21]:
def predict_entities(text):
    labels = ["person", "location", "organization"]
    entities = model.predict_entities(text, labels)
    return entities


In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
# from tqdm import tqdm
# entities = {}

# for _, row in tqdm(df.iterrows()):
#     entities["id"] = row["ID"]
#     entities["id"]["entities"] = predict_entities(row["content"])
%%time
df["entities"] = df["content"].apply(predict_entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [25]:
df[["ID","entities" ]].to_json("entities.json")

In [ ]:
import json 
json_object = json.dumps(entities, indent=4)
 
# Writing to sample.json
with open("entities.json", "w") as outfile:
    outfile.write(json_object)